In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
from docling.datamodel.pipeline_options_vlm_model import ResponseFormat
from pathlib import Path

from archaeo_super_prompt.dataset.load import MagohDataset
from archaeo_super_prompt.pdf_to_text.stream_ocr import process_documents, converter, ollama_vlm_options

# 🦆📃 PDF complete ingestion with Docling preprocessing

We try the young framework Docling and its usage of VLLM on the HuggingFace repositories to achieve thoses tasks:

- document OCR with Italian language analysis (VLLM)
- document chunking with these features:
    - layout-aware
    - smart tokenization

All of these things are possible with incorporating several open ML models into the Docling pipeline

In [ ]:
MAX_SAMPLES_FETCHED = 300
SEED = 0.5

dataset = MagohDataset(MAX_SAMPLES_FETCHED, SEED, True)
_selected_ids = [
    35983, 31298
]
selected_ids = set(_selected_ids)
inputs = dataset.get_files_for_batch(selected_ids)

In [ ]:
inputs["filepath"].tolist()

In [ ]:
def main():
    logging.basicConfig(level=logging.INFO)
    TIMEOUT_PER_PAGE = 60*3
    # Example using the Granite Vision model with Ollama:
    doc_converter = converter(ollama_vlm_options(
        model="granite3.2-vision:latest",
        prompt="OCR the full page for markdown-based processing.",
        # Doctags is only supported by doclings vllm for now
        response_format=ResponseFormat.MARKDOWN,
        allowed_timeout=TIMEOUT_PER_PAGE
    ))
    # results = doc_converter.convert(inputs["filepath"].tolist()[1])
    results = process_documents([Path(p) for p in inputs["filepath"].tolist()],
                                doc_converter,
                                TIMEOUT_PER_PAGE)
    return results

In [ ]:
result = main()

## Inspect the results

We export into markdown the results for display it

In [ ]:
from IPython.display import Markdown
Markdown(result[0].export_to_markdown())

In [ ]:
from docling.datamodel.base_models import ConversionStatus


def is_successful(conv_res):
    return (conv_res.status == ConversionStatus.SUCCESS or
            conv_res.status == ConversionStatus.PARTIAL_SUCCESS)


def filename(conv_res):
    return conv_res.input.file.stem

[{"good": is_successful(r), "name": filename(r)} for r in result]

In [ ]:
from docling.chunking import HierarchicalChunker

chunker = HierarchicalChunker()
chunk_iter = chunker.chunk(dl_doc=result[1].document)
chunks = list(chunk_iter)

In [ ]:
for i, chunk in enumerate(chunks):
    print(f"=== {i} ===")
    txt_tokens = len(chunk.text.rstrip().split(" "))  # tokenizer.count_tokens(chunk.text)
    print(f"chunk.text ({txt_tokens} tokens):\n{chunk.text!r}")

    ser_txt = chunker.contextualize(chunk=chunk)
    ser_tokens = len(ser_txt.rstrip().split(" "))  # tokenizer.count_tokens(ser_txt)
    print(f"chunker.contextualize(chunk) ({ser_tokens} tokens):\n{ser_txt!r}")

    print()

In [ ]:
from IPython.display import Markdown
Markdown(result[1].document.export_to_markdown())

## Hybrid-Chunking

This enable to use a token-aware chunking after a first layout-aware chunking, from a tokenizer built from the embedding model that will be used for the chunk selection, the thesaurus match comparisons, etc.

This has two advantages:
1. The chunks will be recutted more suitably for a procesing by the embedding model
2. The chunks might be recutted more shortly without losing contextual information, even if the layout was originally set of big paragraphs which does not cut the text.

In [ ]:
from docling.chunking import HybridChunker

from docling_core.transforms.chunker.tokenizer.huggingface import HuggingFaceTokenizer
from transformers import AutoTokenizer


EMBED_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"
MAX_TOKENS = 512  # set to a small number for illustrative purposes

tokenizer = HuggingFaceTokenizer(
    tokenizer=AutoTokenizer.from_pretrained(EMBED_MODEL_ID),
    max_tokens=MAX_TOKENS,  # optional, by default derived from `tokenizer` for HF case
)
chunker = HybridChunker(tokenizer=tokenizer)
chunk_iter = chunker.chunk(dl_doc=result[1])

In [ ]:
hybrid_chunk_list = list(chunk_iter)

In [ ]:
for i, chunk in enumerate(hybrid_chunk_list):
    print(f"=== {i} ===")
    txt_tokens = len(chunk.text.rstrip().split(" "))  # tokenizer.count_tokens(chunk.text)
    print(f"chunk.text ({txt_tokens} tokens):\n{chunk.text!r}")

    ser_txt = chunker.contextualize(chunk=chunk)
    ser_tokens = len(ser_txt.rstrip().split(" "))  # tokenizer.count_tokens(ser_txt)
    print(f"chunker.contextualize(chunk) ({ser_tokens} tokens):\n{ser_txt!r}")

    print()

In [ ]:
import functools as fnt
# d = hybrid_chunk_list[0].export_json_dict()
pages = set(
    fnt.reduce(
        lambda acc_lst, item: acc_lst + [p.page_no for p in item.prov],
        hybrid_chunk_list[0].meta.doc_items,
        [],
    )
)
# pages = set(fnt.reduce(lambda acc_lst, item: acc_lst + [p["page_no"] for p in item["prov"]], d["meta"]["doc_items"], []))
print(hybrid_chunk_list[0].meta.doc_items[0].label)